In [1]:
import pymongo
from pymongo import MongoClient
import pandas
import pandas as pd
import phonenumbers as pn
from bson.objectid import ObjectId

In [2]:
def mongo_connect():
    # Try to connect to MongoDB,  exit if not successful.
    try:
        conn = MongoClient('localhost', 27018)
        # print "Connected successfully to MongoDB, instance created!"

    except (pymongo.errors.ConnectionFailure, e):
        print("Could not connect to MongoDB: {}".format(e))

    name = 'SA_Voting_Data'

    # create database on instance
    if name in conn.database_names():
        # print "Found SA_Voiting_Data Database"
        # print "Connecting to db..."
        db = conn[name]  # Create the database

        # create one collection, called conversations
        conversations = db.conversations_collection
        raw = db.raw_collection
        # print ("Connected!")
    else:
        # print "Creating SA_Voting_Data Database"
        db = conn[name]  # create the database
        conversations = db.conversations_collection
        raw = db.raw_collection

    # collection is a collection in the mongodb instance created above
    # db is the database on the mongodb instance
    # conn is the mongodb instance
    return conn, db, conversations, raw

In [3]:
def getMxitFromContent(phonenumber):
    conn, db, conversations, raw = mongo_connect()
    try:
        c = conversations.find({'content': phonenumber}, {'poi': 1})
    finally:
        conn.close()
    return pd.DataFrame(list(c))['poi']

def getConvoByPOI(mxit):
    conn, db, conversations, raw = mongo_connect()
    try:
        c = conversations.find({'poi': mxit})
    finally:
        conn.close()
    return pd.DataFrame(list(c))

def getUNIQUECONTENT():
    conn, db, conversations, raw = mongo_connect()
    try:
        c = conversations.distinct('content', {'poi': {'$regex': '^m'}})
    finally:
        conn.close()
    x = pd.DataFrame(list(c))
    x.columns = ['content']
    return x[x.content.str.isdigit()==True]
    
def getcleanednumbers(phonenumberz):
    cleaned_phonenumbers = []
    wrong = []
    for p,v in phonenumberz.content.iteritems():
        try:
            x = pn.parse(v, "ZA")
            if pn.is_possible_number(x) is True:
                cleaned_phonenumbers.append(['+'+str(x.country_code)+str(x.national_number),v])
            if pn.is_possible_number(x) is False:
                wrong.append(v)
#                 print x,v, pn.is_possible_number(x)
        except:
            pass
    return cleaned_phonenumbers, wrong

In [4]:
phonenumbers = getUNIQUECONTENT()
cleaned_phonenumbers, wrong = getcleanednumbers(phonenumbers)

In [5]:
len(cleaned_phonenumbers)

15461

In [6]:
# count = 0
# progress = 0
# for cleaned_number in cleaned_phonenumbers:
#     ussd = cleaned_number[0]
#     content = cleaned_number[-1]
#     non_mxit_convo = getConvoByPOI(ussd)
#     if len(non_mxit_convo) > 0:
#         count +=1
#         non_mxit_convo.to_csv("./Merge/non_mxit_convo_{}.csv".format(count))
#         mxit = getMxitFromContent(content)
#         mxit.to_csv('./Merge/mxits_{}.csv'.format(count))
#         mxit_convo = getConvoByPOI(mxit[0])
#         mxit_convo.to_csv("./Merge/mxit_convo_{}.csv".format(count))
#     else:
#         pass
#     progress += 1
#     print(progress, len(cleaned_phonenumbers))

In [7]:
def parralelProcess(cleaned_number):
    ussd = cleaned_number[0]
    content = cleaned_number[-1]
    non_mxit_convo = getConvoByPOI(ussd)
    if len(non_mxit_convo) > 0:
        non_mxit_convo.to_csv("./Merge/non_mxit_convo_{}.csv".format(non_mxit_convo.poi[0]))
        mxit = getMxitFromContent(content)
        mxit.to_csv('./Merge/mxits_{}.csv'.format(non_mxit_convo.poi[0]))
        mxit_convo = getConvoByPOI(mxit[0])
        mxit_convo.to_csv("./Merge/mxit_convo_{}.csv".format(non_mxit_convo.poi[0]))
    else:
        pass

In [8]:
from concurrent.futures import ProcessPoolExecutor

with ProcessPoolExecutor() as executor:
    results = list(executor.map(parralelProcess, cleaned_phonenumbers))
    print(results)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

In [ ]:
results

In [ ]:
df = pd.DataFrame(addresses)
df.columns = ['address']
mxit_df = df[df['address'].str.contains(r'^m')==True]

In [ ]:
df['address']

GET RANDOM SAMPLE

In [ ]:
random_sample = df.sample(n=100)

In [ ]:
random_sample

For random sample get conversations

In [ ]:
try:
    c = conversations.find({'poi': '+27725553128'})
finally:
    conn.close()

In [ ]:
sample_conversation = pd.DataFrame(list(c))

In [ ]:
set(sample_conversation['transport_type'])

In [ ]:
for k,v in random_sample['address'].iteritems():
    try:
        c = conversations.find({'poi': v})
    finally:
        conn.close()
    sample_conversation = pd.DataFrame(list(c))
    print set(sample_conversation['transport_type'])

In [ ]:
try:
    c = conversations.distinct('poi', {'poi': {'$regex': r'^m'}, 'transport_type': u'mxit'})
finally:
    conn.close()

In [ ]:
mmm = list(c)

In [ ]:
len(mmm)

In [ ]:
try:
    c = conversations.distinct('poi', {'poi': {'$regex': r'^m'}, 'transport_type': u'mxit'})
finally:
    conn.close()
mmmm = list(c)
print len(mmmm)

In [ ]:
import re
pattern = re.compile("^[^m]*$")
if pattern.match('m+2777'):
    print 'ye'
else:
    print 'fuck'

In [ ]:
try:
    c = conversations.distinct('poi', {'poi': {'$regex': r'^[^m]*$'}, 'transport_type': u'mxit'})
finally:
    conn.close()
interesting = list(c)
print len(interesting)

In [ ]:
try:
    c = conversations.find({'poi': '+27728559465'})
finally:
    conn.close()
t = pd.DataFrame(list(c))
t.to_csv("{}-{}.csv")

In [ ]:
t.columns

In [ ]:
pd.set_option('display.max_colwidth', -1)
t[['routing_metadata','transport_type','to_addr', 'from_addr', 'content', 'in_reply_to']]

In [ ]:
convers

In [ ]:
count = 0
for interest in interesting[0:30]:
    try:
        c = conversations.find({'poi': interest})
    finally:
        conn.close()
    t = pd.DataFrame(list(c))
    count += 1
    print count

    t[['routing_metadata','transport_type','to_addr', 'from_addr', 'content', 'in_reply_to']].to_csv('./interesting/{}_{}.csv'.format('interesting',count))

In [ ]:
try:
    c = conversations.distinct('content', {'poi': {'$regex': '^m'}})
finally:
    conn.close()
interesting = list(c)
print interesting

In [ ]:
ii = pd.DataFrame(interesting)
ii.columns = ['content']
phonenumberz = ii[ii.content.str.isdigit()==True]

In [ ]:
ii.to_csv('./interesting_messages.csv')

In [ ]:
ii.columns = ['content']
phonenumberz = ii[ii.content.str.isdigit()==True]

In [ ]:
phonenumberz = ii[ii.content.str.isdigit()==True]

In [ ]:
for p,v in phonenumberz.content.iteritems():
    print v

In [ ]:
len(ii)

In [ ]:
import phonenumbers
cleaned_phonenumbers = []
wrong = []
for p,v in phonenumberz.content.iteritems():
    try:
        x = phonenumbers.parse(v, "ZA")
        if phonenumbers.is_possible_number(x) is True:
            cleaned_phonenumbers.append(['+'+str(x.country_code)+str(x.national_number),v])
        if phonenumbers.is_possible_number(x) is False:
            wrong.append(v)
            print x,v, phonenumbers.is_possible_number(x)
    except:
        print v

In [ ]:
cleaned_phonenumbers

In [ ]:
for cleaned_phonenumber in cleaned_phonenumbers:
    raw = cleaned_phonenumber[-1]
    phone = cleaned_phonenumber[0]
    ussd = get
    if

In [ ]:
len(wrong)

In [ ]:
temporary = pd.DataFrame(cleaned_phonenumbers)
temporary.columns = ['cleaned_phonenumbers']
temporary.to_csv('./interesting/cleaned_phonenumbers.csv')

In [ ]:
for cleaned_phonenumber in cleaned_phonenumbers[0:10]:
    print cleaned_phonenumber
#     c = conversations.find({'poi': cleaned_phonenumber})
#     print list(c)

In [ ]:
def getMxitFromContent(phonenumber):
    try:
        c = conversations.find({'content': phonenumber}, {'poi': 1})
    finally:
        conn.close()
    return pd.DataFrame(list(c))['poi']

def getConvoByPOI(mxit):
    try:
        c = conversations.find({'poi': mxit})
    finally:
        conn.close()
    return pd.DataFrame(list(c))